configure mysql/python connection and prepare the databse for use

In [1]:
# import password from config file
from config import db_password

In [2]:
# import mysql.connector
import mysql.connector as connector

In [5]:
# connect MySQL with Python
connection = connector.connect(user='root', password = db_password)
print("Connection between MySQL and Python is established.\n")

Connection between MySQL and Python is established.



In [6]:
# create cursor
cursor = connection.cursor()
print("Cursor is created to communicate with the MySQL using Python.\n")

Cursor is created to communicate with the MySQL using Python.



In [7]:
# use the database
cursor.execute('USE little_lemon_db;')
print("The database little_lemon is set for use.\n")

The database little_lemon is set for use.



ESTABLISH POOL CONNECTION 